#TP parte 1

In [ ]:
import pandas as pd
import numpy as np
import requests
from collections import Counter
import plotly.express as px
from matplotlib import pyplot as plt
from pandas_profiling import ProfileReport

In [ ]:
with requests.get(
    "https://docs.google.com/spreadsheets/d/1-DWTP8uwVS-dZY402-dm0F9ICw_6PNqDGLmH0u8Eqa0/export?format=csv"
) as r, open("impuestos.csv", "wb") as f:
    for chunk in r.iter_content():
        f.write(chunk)



conversion de variables
missing
reduccion de dimension
etc

In [ ]:
df = pd.read_csv("impuestos.csv")

In [ ]:
# Usando pandas profiling
report = ProfileReport(df, title='Encuesta de sueldos sysarmy 2020.02', explorative=True, lazy=False)
report

TypeError: ignored